In [5]:
%load_ext autoreload
%autoreload 2

In [42]:
import json 
import numpy as np 
from arcs.setup_functions import ReactionGibbsandEquilibrium
from monty.serialization import loadfn

In [46]:
dft_data = loadfn('../app/data/hse06_dft_data.json')
rge = ReactionGibbsandEquilibrium(dft_data)
gibbs_free_energy = rge.Gibbs(dft_dict=dft_data['CO2'],temperature=100,pressure=1)
gibbs_free_energy


TypeError: ReactionGibbsandEquilibrium.Gibbs() got multiple values for argument 'dft_dict'

In [41]:
reaction = 'CO + H2O = H2 + CO2'
from chempy import Equilibrium 
eq = Equilibrium.from_string(reaction)
rge.reaction_gibbs(reaction=eq,
                   pressure=1,
                   temperature=100)

-0.6184489112909475